In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
%matplotlib inline

In [2]:
train = pd.read_csv("data/train.csv", index_col="PassengerId")
test = pd.read_csv("data/test.csv", index_col="PassengerId")
train.shape, test.shape

((891, 11), (418, 10))

In [3]:
train.head(2)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
label_name = "Survived"

In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(train.drop(label_name, axis=1)).toarray()
X_test = ohe.transform(test).toarray()

X_train.shape

(891, 2080)

In [6]:
from sklearn.preprocessing import OneHotEncoder

y_train = ohe.fit_transform(train[[label_name]])
y_train = y_train.toarray()
print(ohe.get_feature_names_out())

['Survived_0' 'Survived_1']


In [7]:
y_train.shape

(891, 2)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(X_train.shape[1], 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, y_train.shape[1]),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) 
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2080, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [9]:
logits = model(torch.FloatTensor(X_train))
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred[:10]}")

Predicted class: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [10]:
nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    hypothesis = model(torch.FloatTensor(X_train))
#     cost = (y_train * -torch.log(hypothesis)).sum(dim=1).mean()
    cost = F.cross_entropy(hypothesis, torch.FloatTensor(y_train))
    optimizer.zero_grad()
    # gradient
    cost.backward()
    # Weight and bias update
    optimizer.step()

    if epoch % 20 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/100 Cost: 0.694087
Epoch   20/100 Cost: 0.693307
Epoch   40/100 Cost: 0.692549
Epoch   60/100 Cost: 0.691812
Epoch   80/100 Cost: 0.691095
Epoch  100/100 Cost: 0.690397


In [11]:
def test(model, X_test):
    with torch.no_grad():
        output = model(torch.FloatTensor(X_test))
        _, predicted = torch.max(output.data, 1)
        return predicted
    
    
y_predict = test(model, X_test)
y_predict = np.array(y_predict)
y_predict[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])